Projekt R - Prepoznavanje znakova skupova podataka MNIST i Kaggle A-Z pomoću dubokih modela

Kaggle - https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format
MNIST - http://yann.lecun.com/exdb/mnist/

In [25]:
#dodavanje svih potrebnih biblioteka
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sklearn
import sklearn.model_selection
import itertools
import math



Nakon učitavanja potrebnih biblioteka, učitavamo podatke iz MNIST-a i Kaggle-a, pretvaramo ih u numpy arrayeve te spajamo MNIST train i test skupove u jedan skup jer ćemo model trenirati na Kaggle i MNIST skupovima zajedno.

In [ ]:
(x_tr, y_tr), (x_ts, y_ts) = tf.keras.datasets.mnist.load_data()
csvPath = "A_Z Handwritten Data.csv"
x_az = []
y_az = []
for row in open(csvPath):
        row = row.split(",")
        #prvi element u redu je labela slike
        label = int(row[0])
        #svi ostali elementi u redu su pikseli slike, pretvaramo ih u numpy array unsigned 8 bit integera (s obzirom da poprimaju vrijednosti od 0 do 255)
        image = np.array([int(x) for x in row[1:]], dtype = "uint8")
        #preoblikujem image u 28x28 matricu (slike su predstavljene kao red od 784 piksela)
        image = image.reshape((28, 28))
        y_az.append(label)
        x_az.append(image)
        
    #pretvaranje u numpy array radi lakšeg korištenja
y_az = np.array(y_az, dtype = "uint8")
x_az = np.array(x_az, dtype = "uint8")
#spajanje train i test skupa za mnist
x_mnist = np.vstack([x_tr, x_ts])
y_mnist = np.hstack([y_tr, y_ts])

Funkcija za izradu podskupa podataka sa maksimalno n uzoraka po klasi.

In [ ]:
def make_subsets(data, labels, n):
  x_samples = []
  y_samples = []

  # iteriranje po svim klasama
  for class_ in np.unique(labels):
    # array sa indeksima elemenata koji pripadaju klasi class_
    class_indices = np.where(labels== class_)[0]
    # ako ima više od n elemenata u klasi, odaberi n slučajnih elemenata
    if len(class_indices) > n:
      random_indices = np.random.choice(class_indices, size=n, replace=False)
    # ako ima manje od n elemenata u klasi, odaberi sve elemente
    else:
      random_indices = class_indices
    # spajanje odabranih elemenata u listu
    x_samples.append(data[random_indices])
    y_samples.append(labels[random_indices])

  # spajanje u numpy array
  #x_samples = np.concatenate(x_samples)
  #y_samples = np.concatenate(y_samples)
  x_samples = np.vstack(x_samples)
  y_samples = np.hstack(y_samples)

  # pretovrba u uint8 (0-255)
  x_samples = np.array(x_samples, dtype="uint8")
  y_samples = np.array(y_samples, dtype="uint8")
  return (x_samples, y_samples)

Funkcija za izradu podskupa podataka sa maksimalno n uzoraka po klasi za skup podataka MNIST.

Spajamo testni i trening MNIST skup podataka, te Kaggle skup podataka u jedan zajednički skup podataka.

In [ ]:
def concat_datasets(x1, y1, x2, y2):
  x = np.vstack([x1, x2])
  y = np.hstack([y1, y2+10])
  return (x, y)

Preuređujemo podatke tako da budu u skladu s potrebama za daljnju obradu, dakle pretvaramo vrijednosti u interval [0,1], dodajemo dimenziju kako bi bio dobili tenzor odgovarajuće dimenzije, također računamo težine klasa kako bi pri treniranju modela skup podataka bio ravnomjerniji. Te labele pretvaramo u one-hot enkodiranje. Konačno skup podataka se dijeli na trening i test skup.

In [ ]:
def divide_prepare_split(x, y, ratio=0.2):
	#postavi sve vrijednosti između 0 i 1
	x = x / 255

	#pretvaranje u grayscale (dodavanje kanala)
	x = np.expand_dims(x, axis=-1)

	#shuffleanje podataka i labela zajedno sa seedom = 420 i one-hot encoding labela
	x, y = sklearn.utils.shuffle(x, y, random_state=420)
	y = tf.keras.utils.to_categorical(y, num_classes=36)
	print(y.sum(axis=0))

	#izračunjavanje težina za svaku klasu
	classTotals = y.sum(axis=0)
	classWeight = {}
	# iteriranje kroz sve klase i računanje težine
	for i in range(0, len(classTotals)):
		classWeight[i] = classTotals.max() / classTotals[i]

	#podijeli na trening i test set (20% test, 80% train)
	X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(x, y, test_size=ratio, random_state=420, stratify=y)

	return (X_train, X_val, Y_train, Y_val, classWeight)

Konačno slijedi definicija modela, odnosno određivanje slojeva i njihovih parametara. Za model smo odabrali duboki model koji se sastoji od 2 konvolucijska sloja i 2 potpuno povezana sloja. Za aktivacijske funkcije koristimo ReLU, a za optimizaciju Adam, moglo se koristiti i SGD ili bilo koji drugi optimizator.

In [ ]:

def create_model(num_filters, kernel_size, dropout_rate, regularizers):
    if(regularizers == 1):
        regularizers = tf.keras.regularizers.l1(0.01)
    elif(regularizers == 2):
        regularizers = tf.keras.regularizers.l2(0.01)
    elif(regularizers == 0):
        regularizers = None
    model = tf.keras.models.Sequential()
    #dodavanje konvolucijskog sloja od 32 filtra sa 3x3 kernelom, relu aktivacijskom funkcijom, input shape je 28x28x1 (jer je grayscale) i regularizacija L2
    model.add(tf.keras.layers.Conv2D(num_filters, kernel_size, activation='relu', input_shape=(28, 28, 1)))
    #dodavanje max pooling sloja sa 2x2 kernelom, svrha poolinga je smanjiti dimenzionalnost podataka i time smanjiti broj parametara
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    #dodavanje batch normalizacije, svrha batch normalizacije je ubrzati treniranje i smanjiti vanjske utjecaje na model, dakle oduzima se srednja vrijednost i dijeli se standardnom devijacijom
    model.add(tf.keras.layers.BatchNormalization())
    #dodavanje dropout sloja, svrha dropouta je sprečavanje overfittinga, odnosno slučajno isključivanje neurona (20% neurona u ovom slučaju)
    model.add(tf.keras.layers.Dropout(dropout_rate))
    #dodavanje konvolucijskog sloja od 64 filtra sa 3x3 kernelom, relu aktivacijskom funkcijom i regularizacija L2
    model.add(tf.keras.layers.Conv2D(num_filters*2, kernel_size, activation='relu', kernel_regularizer=regularizers))
     #dodavanje max pooling sloja sa 2x2 kernelom
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    #dodavanje batch normalizacije
    model.add(tf.keras.layers.BatchNormalization())
    #dodavanje flatten sloja, flatten sloj pretvara matricu u vektor
    model.add(tf.keras.layers.Flatten())
    #dodavanje potpuno povezanog sloja od 64 neurona i relu aktivacijskom funkcijom
    parameter = (num_filters*2*(((((28-(kernel_size-1))/2)-(kernel_size-1))/2)**2))
    model.add(tf.keras.layers.Dense(parameter, activation='relu'))
    #dodavanje potpuno povezanog sloja od 36 neurona i softmax aktivacijskom funkcijom, softmax aktivacijska funkcija vraća vjerojatnosti za svaku klasu
    model.add(tf.keras.layers.Dense(36, activation='softmax'))
    #kompajliranje modela, koristi se adam optimizator, categorical_crossentropy kao loss funkcija i accuracy kao metrika
    model.compile(optimizer=tf.keras.optimizers.Adam(      
                learning_rate=0.0001,
    ),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    #sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(optimizer=sgd,
    #            loss='categorical_crossentropy',
    #            metrics=['accuracy'])
    #vraćanje modela
    return model

Kako gore nismo naveli parametre modela, parametre modela optimiziramo pomoću grid searcha. Za svaki od parametara modela određujemo raspon vrijednosti, a zatim za svaku kombinaciju parametara modela treniramo model i određujemo točnost modela na test skupu. Na kraju odabiremo parametre modela koji daju najbolju točnost.

In [ ]:
size_of_class = 100000 #oko 60000 je najveća klasa, dakle ovo uzima cijeli dataset
xm, ym = make_subsets(x_mnist, y_mnist, size_of_class)
xa, ya = make_subsets(x_az, y_az, size_of_class)
(x, y) = concat_datasets(xm, ym, xa, ya)
(X_train, X_val, Y_train, Y_val, classWeight) = divide_prepare_split(x, y, 0.2)
num_filters = [16, 32]
kernel_size = [3, 5]
dropout_rate = [0.2, 0.4]
regularizers = [1, 2]
param_grid = list(itertools.product(num_filters, kernel_size, dropout_rate, regularizers))

#grid search
scores = []
for params in param_grid:
    print(params)
    model = create_model(params[0], params[1], params[2], params[3])
    model.fit(X_train, Y_train, epochs=15, batch_size=128, class_weight=classWeight, validation_data=(X_val, Y_val))
    scores.append(model.evaluate(X_train, Y_train, verbose=0))
    
#best_index = np.argmin([x[0] for x in scores]) #za loss
best_index = np.argmax([x[1] for x in scores]) #za accuracy
best_num_filters, best_kernel_size, best_dropout_rate, best_regularizers = param_grid[best_index]

Konačno možemo istrenirati model s optimalnim parametrima.

In [ ]:
best_model = create_model(best_num_filters, best_kernel_size, best_dropout_rate, best_regularizers)
best_history = model.fit(X_train, Y_train, epochs=15, batch_size=128, class_weight=classWeight, validation_data=(X_val, Y_val))

loss, accuracy = best_model.evaluate(X_val, Y_val, verbose=0)
print("Test loss: ", loss)
print("Test accuracy: ", accuracy)
best_model.save('model.h5')

Nad modelom je moguće izvesti unakrsnu validaciju kako bi se odredila kvaliteta generalizacije modela. Za unakrsnu validaciju koristimo k-struku validaciju, gdje k predstavlja broj podskupova na koje se podijeli skup podataka. Za svaki od podskupova model se trenira na ostalim podskupovima, a testira se na trenutnom podskupu.

In [ ]:
splits = 5
kfold = sklearn.model_selection.KFold(n_splits=splits)
#k-struka unakrsna validacija
fold = 0
fold_stats = []
for train, val in kfold.split(X_train, Y_train):
    print(fold)
    model_cv = create_model(best_num_filters, best_kernel_size, best_dropout_rate, best_regularizers)
    model_cv.fit(X_train[train], Y_train[train], epochs=15, batch_size=128, class_weight=classWeight, validation_data=(X_train[val], Y_train[val]))
    fold_stats.append(model_cv.evaluate(X_train[val], Y_train[val], verbose=0))
    fold += 1
    
mean_loss = np.mean([x[0] for x in fold_stats])
std_loss = np.std([x[0] for x in fold_stats])
mean_accuracy = np.mean([x[1] for x in fold_stats])
std_accuracy = np.std([x[1] for x in fold_stats])
print("Mean loss: ", mean_loss)
print("Std loss: ", std_loss)
print("Mean accuracy: ", mean_accuracy)
print("Std accuracy: ", std_accuracy)


Evaluacija modela na potpunom skupu podataka (trening i test skup podataka).

In [ ]:
#spajanje mnist i az
x_eval_conc = np.vstack([x_mnist, x_az])
y_eval_conc = np.hstack([y_mnist, y_az+10])

#prilagodba podataka u tenzore odgovarajućih dimenzija
x_eval = np.expand_dims(x_eval_conc, axis=-1)
y_eval = tf.keras.utils.to_categorical(y_eval_conc, num_classes=36)

#evaluacija na svim podatcima
loss, accuracy = model.evaluate(x_eval, y_eval, verbose=0)
print("Eval loss: ", loss)
print("Eval accuracy: ", accuracy)


Konačno možemo prikazati rezultate modela na trening i testnom skupu u obliku grafa loss-a i accuracy-a po epohama.

In [ ]:
#model = create_model(best_num_filters, best_kernel_size, best_dropout_rate, best_regularizers)
#best_history = model.fit(X_train, Y_train, epochs=20, batch_size=128, class_weight=classWeight, validation_data=(X_val, Y_val))

#Plotanje loss-a i accuracy-a za train i validation skupove
plt.plot(best_history.history['loss'])
plt.plot(best_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(best_history.history['accuracy'])
plt.plot(best_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#print(best_history.history['loss'])
#print(best_history.history['val_loss'])
#print(best_history.history['accuracy'])
#print(best_history.history['val_accuracy'])


Usporedba modela s najvišim accuracy-om i modela s najnižim loss-om u grid search-u.

In [ ]:
#print(scores)
#print(np.argmin([x[0] for x in scores]))
#print(np.argmax([x[1] for x in scores]))

#crtanje grafa za loss i accuracy za train i validation skupove
plt.plot(history_list[np.argmin([x[0] for x in scores])].history['accuracy'])
plt.plot(history_list[np.argmin([x[0] for x in scores])].history['val_accuracy'])
plt.plot(history_list[np.argmax([x[1] for x in scores])].history['accuracy'])
plt.plot(history_list[np.argmax([x[1] for x in scores])].history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation', 'train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_list[np.argmin([x[0] for x in scores])].history['loss'])
plt.plot(history_list[np.argmin([x[0] for x in scores])].history['val_loss'])
plt.plot(history_list[np.argmax([x[1] for x in scores])].history['loss'])
plt.plot(history_list[np.argmax([x[1] for x in scores])].history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation', 'train', 'validation'], loc='upper left')
plt.show()
